In [1]:
# -*- coding: utf-8 -*-
"""
Created on 2019/9/29 下午 05:17
@author: Ivan Y.W.Chiu

python ./practice/cifar10_keras.py train --logs=./practice/logs
python ./practice/cifar10_keras.py train --logs=./practice/logs --augmentation=1
python ./practice/cifar10_keras.py train --logs=./practice/logs --augmentation=1 --resume=1 --weights=./practice/logs/D20190929T2220/cifar10_test_epoch020.h5
python ./practice/cifar10_keras.py detect --logs=./practice/logs --weights=./practice/logs/D20191001T0153/cifar10_ResNet56v2_epoch003.h5

"""
import os, sys, time
ROOT_PATH = r"/media/yuwei/Data/YuWei/ML_DL/_Courses/HungYiLee/ML_2019_HW"
os.chdir(ROOT_PATH)
sys.path.append(ROOT_PATH)


from keras.datasets import cifar10
# from practice.src import model as modellib
import practice.src.model as modelib
import practice.src.utils as utils
from practice.src.config import Config
import numpy as np

DEFAULT_LOG_PATH = os.path.join(ROOT_PATH, "hw3", "logs")

############################################################
#  Session Setting
############################################################
# If you face the error about convolution layer,
# use this block to enable the memory usage of GPU growth.
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=tf_config)
set_session(sess)  # set this TensorFlow session as the default session for Keras


Using TensorFlow backend.


In [2]:
############################################################
#  Configurations
############################################################
class Cifar10Config(Config):
    # ResNetv2 depth (9n+2): 
    RESNET_DEPTH = 20

    # The name of your work:
    NAME = 'cifar10_ResNet{:s}v2'.format(str(RESNET_DEPTH))

    # Number of class in dataset:
    N_CLASS = 10

    # The shape of image in dataset:
    IMG_SHAPE = [32, 32, 3]

    # Learning rate:
    LR = 0.001

    # Epochs:
    EPOCHS = 3

    # Training batch size:
    BATCH_SIZE = 256

    # Validation rate:
    VALIDATION_RATE = 0

config = Cifar10Config()
config_list = config.display(print_out=True)

Configurations:
ACTIVATION_FUNC                relu
BATCH_SIZE                     256
BIAS_INIT_DEFAULT              0
CONV2D_KERNEL_SIZE             (3, 3)
CONV2D_PADDING                 SAME
CONV2D_STRIDES                 1
DROPOUT_RATE                   0
EPOCHS                         3
IMG_SHAPE                      [32, 32, 3]
KERNEL_INIT_METHOD             glorot_normal
LR                             0.001
MAXPOOL2D_KERNEL_SIZE          (2, 2)
MAXPOOL2D_PADDING              SAME
MAXPOOL2D_STRIDES              2
NAME                           cifar10_ResNet20v2
N_CLASS                        10
RESNET_DEPTH                   20
SAVE_BEST_ONLY                 False
SUBTRACT_PIXEL_MEAN            True
UPSAMPLING_KERNEL_SIZE         2
UPSAMPLING_METHOD              nearest
VALIDATION_RATE                0


In [3]:
model = modelib.NNModel(mode="training", config=config, logdir=DEFAULT_LOG_PATH, resume=0)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
############################################################
#  Datasets
############################################################
class Cifar10Dataset(utils.Dataset):
    def feed_config(self, f_config):
        self.config = f_config
        self.n_class = f_config.N_CLASS
        self.img_shape = f_config.IMG_SHAPE
        self.val_rate = f_config.VALIDATION_RATE

    def load(self, usage):
        """
        Load the data (image, label) from dataset directory.
        :param usage: 'train' or 'detect'
        :return:
        """
        if usage == 'train':
            (self.x_data, self.y_data), _ = cifar10.load_data()
        else:
            _, (self.x_data, self.y_data) = cifar10.load_data()

dataset = Cifar10Dataset()
dataset.feed_config(config)
dataset.load(usage='train')
dataset.prepare()

In [5]:
dataset.use_val

True

In [6]:
dataset.split()

Training set: 40000
Validation set: 10000
